# <div align="center">Spotify Playlist Generator </div>

## Objective

The purpose of this project was to create a spotify playlist 


In [120]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

cid = '362ab65e27f44f56b2bd2b2f62987e12'
secret = '3a3c7171323649d0a3f3e2b8e57283ab'
username = 'maelstromcarnage'

scope = 'user-top-read user-library-read playlist-modify-public playlist-read-private'

redirect_uri='http://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [128]:
import pandas as pd
import numpy as np

results = sp.current_user_top_tracks(limit=50, offset=0,time_range='long_term')

artists = []
albums = []
track_names = []
track_ids = []
durations = []
popularity = []

for i, item in enumerate(results['items']):
    artists.append(item['artists'][0]['name'])
    albums.append(item['album']['name'])
    track_names.append(item['name'])
    track_ids.append(item['id'])
    durations.append(item['duration_ms'])
    popularity.append(item['popularity'])
    
df_topfifty = pd.DataFrame({
    "Track Name": track_names,
    "Artist": artists,
    "Album": albums,
    "Duration [hh:mm:ss]": pd.to_timedelta(durations, unit='ms').astype(str).map(lambda x: x[7:]),
    "Popularity": popularity
})

df_topfifty.head()

,Track Name,Artist,Album,Duration [hh:mm:ss],Popularity
0,before I close my eyes,XXXTENTACION,?,00:01:39.658000,68
1,See You Again (feat. Kali Uchis),"Tyler, The Creator",Flower Boy,00:03:00.386000,80
2,CALLIGRAPHY,Saba,CARE FOR ME,00:03:04.250000,56
3,SleepMode,BONES,SoftwareUpdate1.0,00:02:43.512000,53
4,hate will never win,XXXTENTACION,A GHETTO CHRISTMAS CAROL,00:01:44.071000,59


In [127]:
features = []
for i in range(0,len(track_names)):
    audio_features = sp.audio_features(track_ids[i])
    for track in audio_features:
        features.append(track)

df_topfifty_features = pd.DataFrame(features, index = track_names)

# Mode is an unnecessary feature for our analysis
del df_topfifty_features['mode']

df_topfifty_features.head()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
before I close my eyes,0.356,0.143,9,-15.148,0.0388,0.976,0.000019,0.1110,0.3340,75.522,audio_features,16x9bQHw8TBNaGDyULvwad,spotify:track:16x9bQHw8TBNaGDyULvwad,https://api.spotify.com/v1/tracks/16x9bQHw8TBN...,https://api.spotify.com/v1/audio-analysis/16x9...,99658,4
See You Again (feat. Kali Uchis),0.558,0.559,6,-9.222,0.0959,0.371,0.000007,0.1090,0.6200,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
CALLIGRAPHY,0.713,0.533,3,-9.713,0.3150,0.732,0.000947,0.1110,0.3060,73.973,audio_features,7LBdv0ek7Tj2s0bgxmlrC6,spotify:track:7LBdv0ek7Tj2s0bgxmlrC6,https://api.spotify.com/v1/tracks/7LBdv0ek7Tj2...,https://api.spotify.com/v1/audio-analysis/7LBd...,184250,4
SleepMode,0.397,0.248,0,-12.878,0.0326,0.550,0.003780,0.0954,0.0976,106.017,audio_features,6fyFiyoywfLYikFDQuabmH,spotify:track:6fyFiyoywfLYikFDQuabmH,https://api.spotify.com/v1/tracks/6fyFiyoywfLY...,https://api.spotify.com/v1/audio-analysis/6fyF...,163512,4
hate will never win,0.648,0.218,10,-13.603,0.3840,0.817,0.005000,0.1080,0.5960,88.577,audio_features,0fWkWSpXp1V8nWslbA25Sg,spotify:track:0fWkWSpXp1V8nWslbA25Sg,https://api.spotify.com/v1/tracks/0fWkWSpXp1V8...,https://api.spotify.com/v1/audio-analysis/0fWk...,104072,4
